In [1]:
import torch

# List available models
torch.hub.list('pytorch/fairseq')

Using cache found in /home/vzhekova/.cache/torch/hub/pytorch_fairseq_main


['bart.base',
 'bart.large',
 'bart.large.cnn',
 'bart.large.mnli',
 'bart.large.xsum',
 'bpe',
 'camembert',
 'camembert-base',
 'camembert-base-ccnet',
 'camembert-base-ccnet-4gb',
 'camembert-base-oscar-4gb',
 'camembert-base-wikipedia-4gb',
 'camembert-large',
 'camembert.v0',
 'conv.stories',
 'conv.stories.pretrained',
 'conv.wmt14.en-de',
 'conv.wmt14.en-fr',
 'conv.wmt17.en-de',
 'data.stories',
 'dynamicconv.glu.wmt14.en-fr',
 'dynamicconv.glu.wmt16.en-de',
 'dynamicconv.glu.wmt17.en-de',
 'dynamicconv.glu.wmt17.zh-en',
 'dynamicconv.no_glu.iwslt14.de-en',
 'dynamicconv.no_glu.wmt16.en-de',
 'fastspeech2-en-200_speaker-cv4',
 'fastspeech2-en-ljspeech',
 'gottbert-base',
 'lightconv.glu.wmt14.en-fr',
 'lightconv.glu.wmt16.en-de',
 'lightconv.glu.wmt17.en-de',
 'lightconv.glu.wmt17.zh-en',
 'lightconv.no_glu.iwslt14.de-en',
 'lightconv.no_glu.wmt16.en-de',
 'roberta.base',
 'roberta.large',
 'roberta.large.mnli',
 'roberta.large.wsc',
 's2t_transformer_l-en-asr-librispeech',
 's

In [2]:
# Note: WMT'19 models use fastBPE instead of subword_nmt
# Load an En-Fr Transformer model trained on WMT'14 data:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt14.en-fr', tokenizer='moses', bpe='subword_nmt')
en2de.eval()  # disable dropout

# Access the underlying TransformerModel
assert isinstance(en2de.models[0], torch.nn.Module)

Using cache found in /home/vzhekova/.cache/torch/hub/pytorch_fairseq_main
2022-11-25 11:57:27 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/models/wmt14.en-fr.joined-dict.transformer.tar.bz2 not found in cache, downloading to /tmp/tmphmqsorl3
100%|██████████████████████████████████████████████████████████████| 2316140317/2316140317 [01:51<00:00, 20854854.40B/s]
2022-11-25 11:59:20 | INFO | fairseq.file_utils | copying /tmp/tmphmqsorl3 to cache at /home/vzhekova/.cache/torch/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae
2022-11-25 12:00:09 | INFO | fairseq.file_utils | creating metadata file for /home/vzhekova/.cache/torch/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae
2022-11-25 12:00:09 | INFO | fairseq.file_utils | removing temp file /tmp/tmphmqsorl3
2022-11-25 12:0

In [3]:
# Move model to GPU for faster translation
en2de.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(44512, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): Linear

In [4]:
# Translate a sentence
en2de.translate('Hello world!')
# 'Hallo Welt!'

'Bonjour à tous !'

In [5]:
# Batched translation
en2de.translate(['Hello world!', 'The cat sat on the mat.'])
# ['Hallo Welt!', 'Die Katze saß auf der Matte.']

['Bonjour à tous !', 'Le chat était assis sur le tapis.']

In [95]:
%cd /export/data4/vzhekova/biases-data

/export/data4/vzhekova/biases-data


In [96]:
!echo -e "\nFirst lines of English:\n"
!head data.en-fr.en
!echo -e "\nFirst lines of French:\n"
!head data.en-fr.fr


First lines of English:

﻿Now, I thought, "How could I really capture this?
I mean, from this entry, it would seem that I was born into a world that perceived someone like me to have nothing positive whatsoever going for them, when in fact, today I'm celebrated for the opportunities and adventures my life has procured.
So, I immediately went to look up the 2009 online edition, expecting to find a revision worth noting.
His name was Dr. Pizzutillo, an Italian American, whose name, apparently, was too difficult for most Americans to pronounce, so he went by Dr. P. And Dr. P always wore really colorful bow ties and had the very perfect disposition to work with children.
And, one day, he came in to my session — exhaustive and unforgiving, these sessions — and he said to me, "Wow.
Now, of course, this was a simple ploy on Dr. P's part to get me to do the exercises I didn't want to do before the prospect of being the richest five-year-old in the second floor ward, but what he effectively di

In [97]:
# Tokenize text
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

mt = MosesTokenizer(lang='en')

with open('data.en-fr.en', encoding='utf8') as fin, open('data.en-fr.tok.en','w', encoding='utf8') as fout:
    for line in fin:
        tokens = mt.tokenize(line, return_str=True)
        print(tokens, end='\n', file=fout) 
        
mt = MosesTokenizer(lang='fr')

with open('data.en-fr.fr', encoding='utf8') as fin, open('data.en-fr.tok.fr','w', encoding='utf8') as fout:
    for line in fin:
        tokens = mt.tokenize(line, return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished tokenizing.')

Finished tokenizing.


In [103]:
# Dividing tokenized text into subword units

!subword-nmt learn-bpe -s 10000 < data.en-fr.tok.en > sw.model.en
!subword-nmt apply-bpe -c sw.model.en < data.en-fr.tok.en > sw.data.en-fr.en

!subword-nmt learn-bpe -s 10000 < data.en-fr.tok.fr > sw.model.fr
!subword-nmt apply-bpe -c sw.model.fr < data.en-fr.tok.fr > sw.data.en-fr.fr

print('Finished subword.')

 50%|##################5                  | 5016/10000 [00:10<00:10, 471.65it/s]
Finished subword.


In [104]:
!echo -e "\nFirst lines of tokenized English:\n"
!head sw.data.en-fr.en

!echo -e "\nFirst lines of tokenized French:\n"
!head sw.data.en-fr.fr


First lines of tokenized English:

﻿ Now , I thought , &quot; How could I really capture this ?
I mean , from this ent@@ ry , it would seem that I was born into a world that percei@@ ved someone like me to have nothing positive whatsoever going for them , when in fact , today I &apos;m celebr@@ ated for the oppor@@ t@@ uni@@ ties and adventures my life has pro@@ cur@@ ed .
So , I immediately went to look up the 200@@ 9 online e@@ dition , expec@@ ting to find a re@@ vision wor@@ th n@@ oting .
His name was Dr. P@@ i@@ zz@@ u@@ til@@ lo , an Italian American , whose name , apparently , was too difficult for most Americ@@ ans to pr@@ on@@ ounce , so he went by Dr. P@@ . And Dr. P always wor@@ e really colorful bow ties and had the very perfect disp@@ os@@ i@@ tion to work with children .
And , one day , he came in to my session — exha@@ us@@ tive and un@@ for@@ giving , these sessions — and he said to me , &quot; W@@ ow .
Now , of course , this was a simple p@@ loy on Dr. P &apos;s part

In [105]:
# Binarize the data for training; test with moses tokenizer and subword_nmt failed -> BLEU score of 4
!fairseq-preprocess \
    --source-lang en \
    --target-lang fr \
    --testpref sw.data.en-fr \
    --srcdict wmt14.en-fr.fconv-py/dict.en.txt \
    --tgtdict wmt14.en-fr.fconv-py/dict.fr.txt \
    # --tokenizer	moses \
    # --bpe subword_nmt \
    --destdir data-bin \
    --thresholdtgt 0 \
    --thresholdsrc 0 \
    --workers 8

2023-01-13 12:32:38 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='wmt14.en-

In [106]:
%cd /export/data4/vzhekova/biases-data

/export/data4/vzhekova/biases-data


In [113]:
# Generate translations
!fairseq-generate data-bin  \
    --task translation \
    --source-lang en \
    --target-lang fr \
    --path wmt14.en-fr.fconv-py/model.pt \
    --beam 1 \
    --batch-size 256 \
    --post-process \
    --remove-bpe=subword_nmt > en-fr.decode.log

2023-01-13 15:12:58 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

In [114]:
# Extract the hypotheses and references from the decoding log file
!grep ^H en-fr.decode.log | sed 's/^H-//g' | cut -f 3 | sed 's/ ##//g' > ./hyp.txt
!grep ^T en-fr.decode.log | sed 's/^T-//g' | cut -f 2 | sed 's/ ##//g' > ./ref.txt

!head ./hyp.txt
print("..........")
!head ./ref.txt

Je voulais ouvrir ces jeux comme un allié de la communauté LGBTQ .
Au début de mon séjour là @-@ bas en 2000 , j&apos; ai été très intéressé par les communautés .
Après ça , j&apos; ai été un peu de la bonne façon d&apos; aller à ma prochaine date de zero .
Je suis allé à la maison de la femme pour la célébrer et la congratuler .
Et je pensais que c&apos; était super cool , alors je l&apos; ai montré à mon ami .
Je m&apos; appelle Dan , je suis un partenaire dans une consultance créative mondiale .
Je suis retourné à mon laboratoire à Boston et j&apos; ai fait une petite expérience .
Et le juge l&apos; a cerné comme un adulte , mais je vois cela .
Ici , nous avons une jeune femme et un homme qui jouent à un jeu d&apos; aumôniers .
J&apos; ai eu le grand privilège , lorsque j&apos; étais un jeune avocat , de rencontrer des membres du Groupe des parcs .
..........
Je <<unk>> ouvrir ces portes en tant qu&apos; alliée de la communauté LGBTQ .
À mes débuts en 2000 , j&apos; étais très intér

In [115]:
# Evaluate the model; BLEU score of 26.7 (beam=5), BLEU score of 26.2 (beam=1)
!cat ./hyp.txt | sacrebleu ref.txt

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
{
 "name": "BLEU",
 "score": 26.2,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "55.8/36.3/26.2/18.8 (BP = 0.828 ratio = 0.841 hyp_len = 44280 ref_len = 52639)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}
